In [1]:
import os
import numpy as np
import math
# function of read data
def _read_txt_(url):
    file = open(url, 'r', encoding='utf-8')
    seg_list = []
    lines = file.readlines()
    for i in range(len(lines)):
        seg_list.append(lines[i].rstrip("\n"))
        pass
    file.close()
    return seg_list

emit, trans, context = {},{},{}
possible_tags = []
#init
context["<s>"] = 0
context["</s>"] = 0

train_list = _read_txt_("wiki-en-train.norm_pos")

In [2]:
def _unique_list_(data):
    unique_word_frequency = {}
    for _, seq in enumerate(data):
        wordtags = seq.split(" ")
        for _, wordtag in enumerate(wordtags):
            word = wordtag.split("_")[0]
            if word not in unique_word_frequency:
                unique_word_frequency[word] = 1
            else:
                num = unique_word_frequency[word]
                unique_word_frequency[word] = num + 1
    return unique_word_frequency

def _UNK_(data, refer_dict):
    for idx, seq in enumerate(data):
        new_seq = ""
        wordtags = seq.split(" ")
        for m, wordtag in enumerate(wordtags):
            word, tag = wordtag.split("_")[0], wordtag.split("_")[1]
            if word != "UNK" and refer_dict[word] < 3:
                new = "UNK_" + tag
                new_seq = new_seq + new + " "
            else:
                new_seq = new_seq + wordtag + " "
        data[idx] = new_seq.rstrip(" ")
    return data
unique_word_frequency = _unique_list_(train_list)
train_list = _UNK_(train_list, unique_word_frequency)

In [3]:
# train model
for seq in train_list:
    previous = "<s>"
    context[previous] += 1
    wordtags = seq.split(" ")
    for wordtag in wordtags:
        tmp = wordtag.split("_")
        word, tag = tmp[0], tmp[1]
        if tag not in possible_tags:
            possible_tags.append(tag)
        # count the transtition
        tmp_t = previous + " " + tag
        if tmp_t not in trans.keys():
            trans[tmp_t] = 1
        else:
            trans[tmp_t] += 1
        # count the context
        if tag not in context.keys():
            context[tag] = 1
        else:
            context[tag] += 1
        # count the emission
        tmp_e = tag + " " + word
        if tmp_e not in emit.keys():
            emit[tmp_e] = 1
        else:
            emit[tmp_e] += 1
        previous = tag
    tmp_end = previous + " </s>"
    context["</s>"] +=1
    if tmp_end not in trans.keys():
        trans[tmp_end] = 1
    else:
        trans[tmp_end] += 1

In [4]:
# test
trans_keys = list(trans.keys())
emit_keys = list(emit.keys())
test_list = _read_txt_("wiki-en-test.norm_pos")

In [59]:
# p_example
#-math.log(trans["JJ NN"]/context["NN"])
#-math.log(emit["NN language"]/context["NN"])
N = 0
for seq in test_list:
    wordtags = seq.split(" ")
    N = N + len(wordtags)
denominator = 0
numerator = 0
for y, seq in enumerate(test_list):
    best_score, best_edge = {},{}
    best_score["0 <s>"] = 0
    best_edge["0 <s>"] = None
    wordtags = seq.split(" ")
    real_tags = []
    for idx, wordtag in enumerate(wordtags):
        if idx == len(wordtags) - 1: break
        tmp = wordtag.split("_")
        word, tag = tmp[0], tmp[1]
        if word not in unique_word_frequency:
            word = "UNK"
        real_tags.append(tag)
        for i in range(len(possible_tags)):
            previous = possible_tags[i]
            if idx == 0 and i > 0:
                break
            for next_tag in possible_tags:
                if idx == 0:
                    previous = "<s>"    
                    best_score_value = 0
                else:
                    if str(idx) + " " + previous in best_score.keys():
                        best_score_value = best_score[str(idx) + " " + previous]
                    else:
                        continue
                if previous + " " + next_tag in trans_keys:
                    if next_tag + " " + word in emit_keys:
                        score = best_score_value -\
                        math.log(trans[previous + " " + next_tag]/context[next_tag]) -\
                        math.log(emit[next_tag + " " + word]/context[next_tag])
                    else:
                        score = best_score_value -\
                        math.log(trans[previous + " " + next_tag]/context[next_tag]) -\
                        math.log(1/N)
                    if str(idx+1)+" "+next_tag not in best_score.keys():
                        best_score[str(idx+1)+" "+next_tag] = score
                        best_edge[str(idx+1)+" "+next_tag] = word+" " + previous
                    else:
                        value = best_score[str(idx+1)+" "+next_tag]
                        if score < value:
                            best_score[str(idx+1)+" "+next_tag] = score
                            best_edge[str(idx+1)+" "+next_tag] = word+" " + previous
    tmp = wordtags[-1].split("_")
    word, tag = tmp[0], tmp[1]
    if word not in unique_word_frequency:
        word = "UNK"
    real_tags.append(tag)
    next_tag = "."
    tmp_score = 10000
    tmp_edge = None
    for previous in possible_tags:
        if str(len(wordtags)-1) + " " + previous in best_score.keys():
            best_score_value = best_score[str(len(wordtags)-1) + " " + previous]
        else:
            continue
        if (previous + " " + next_tag) in trans_keys:
            if next_tag + " " + word in emit_keys:
                score = best_score_value -\
                math.log(trans[previous + " " + next_tag]/context[next_tag]) -\
                math.log(emit[next_tag + " " + word]/context[next_tag])
            else:
                score = best_score_value -\
                math.log(trans[previous + " " + next_tag]/context[next_tag]) -\
                math.log(1/N)
            if score < tmp_score:
                tmp_score = score
                tmp_edge = word+" " + previous
    best_score['/ .'] = score
    best_edge["/ ."] = tmp_edge
    
    tags = []
    next_edge = ""
    for i in range(len(wordtags)):
        if i == 0:
            next_edge= "/ ."
        if best_edge[next_edge] == None:break
        tags.append(next_edge.split(" ")[1])
        next_edge = str(len(wordtags)-i-1) + " " + best_edge[next_edge].split(" ")[1]
    tags = list(reversed(tags))
    for i in range(len(tags)):
        denominator +=1
        tag = tags[i]
        real_tag = real_tags[i]
        if tag == real_tag:
            numerator += 1
print(numerator)
print(denominator)

3838
4563


In [60]:
print("acc: " + str(round(numerator/denominator,3)*100) + "%")

acc: 84.1%
